# Covid19 


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.subplots as ps
from prophet import Prophet
from sklearn.metrics import r2_score



# --------------------------------------------------
# $$
datas_confirmed=pd.read_csv("./time_series_covid_19_confirmed.csv", header=None)
datas_death=pd.read_csv("./time_series_covid_19_deaths.csv", header=None)

datas_confirmed.isna().sum()

# 역치 시키기
datas_confirmed=datas_confirmed.T
datas_death=datas_death.T

# # 첫 번째 행을 열 이름으로 설정 (country/regionn에 해당)
datas_confirmed.columns = datas_confirmed.iloc[1]
datas_death.columns = datas_death.iloc[1]


# # 첫 번째 행 제거(province/state 제거)
datas_confirmed = datas_confirmed[2:]
datas_death = datas_death[2:]

datas_confirmed


1,Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2,Lat,33.93911,41.1533,28.0339,42.5063,-11.2027,17.0608,-38.4161,40.0691,-35.4735,...,55.3781,-32.5228,41.377491,-15.3767,6.4238,14.058324,31.9522,15.552727,-13.133897,-19.015438
3,Long,67.709953,20.1683,1.6596,1.5218,17.8739,-61.7964,-63.6167,45.0382,149.0124,...,-3.436,-55.7658,64.585262,166.9592,-66.5897,108.277199,35.2332,48.516388,27.849332,29.154857
4,1/22/20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1/23/20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
6,1/24/20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,5/25/21,66903,132229,127361,13664,32933,1258,3586736,222139,124,...,4467310,271859,98849,4,226136,5931,306334,6670,93428,38706
494,5/26/21,67743,132244,127646,13671,33338,1258,3622135,222269,124,...,4470297,276435,99064,4,227407,6086,306795,6688,93627,38819
495,5/27/21,68366,132264,127926,13682,33607,1258,3663215,222409,124,...,4473677,280372,99344,4,228828,6356,306795,6696,93947,38854
496,5/28/21,69130,132285,128198,13693,33944,1259,3702422,222513,124,...,4477705,282198,99580,4,230147,6396,307569,6723,94430,38918


In [3]:
import pandas as pd
import plotly.express as px

# 데이터 로드
datas_confirmed = pd.read_csv("/mnt/data/time_series_covid_19_confirmed.csv", header=None)
datas_death = pd.read_csv("/mnt/data/time_series_covid_19_deaths.csv", header=None)

# 데이터 전치
datas_confirmed = datas_confirmed.T
datas_death = datas_death.T

# 첫 번째 행을 열 이름으로 설정 (country/region에 해당)
datas_confirmed.columns = datas_confirmed.iloc[1]
datas_death.columns = datas_death.iloc[1]

# 첫 번째 행 제거(province/state 제거)
datas_confirmed = datas_confirmed[2:]
datas_death = datas_death[2:]

# 위도와 경도가 비어있는 경우 해당 나라 삭제
datas_confirmed = datas_confirmed.dropna(subset=[0, 1])
datas_death = datas_death.dropna(subset=[0, 1])

# 날짜 정보 추출
dates = datas_confirmed.index[2:]

# 데이터 형태 변환
datas_confirmed.iloc[2:, :] = datas_confirmed.iloc[2:, :].astype(float)
datas_death.iloc[2:, :] = datas_death.iloc[2:, :].astype(float)

# 위도, 경도, 나라 이름 가져오기
latitudes = datas_confirmed.iloc[0].values
longitudes = datas_confirmed.iloc[1].values
countries = datas_confirmed.columns[2:]

# 데이터를 새로운 형태로 변환
data_confirmed_long = pd.melt(datas_confirmed[2:], id_vars=[0, 1], var_name='Country', value_name='Confirmed')
data_confirmed_long['Date'] = dates.repeat(len(countries)).values

data_death_long = pd.melt(datas_death[2:], id_vars=[0, 1], var_name='Country', value_name='Deaths')
data_death_long['Date'] = dates.repeat(len(countries)).values

# 위도, 경도 정보 추가
data_confirmed_long['Lat'] = data_confirmed_long[0].astype(float)
data_confirmed_long['Long'] = data_confirmed_long[1].astype(float)
data_death_long['Lat'] = data_death_long[0].astype(float)
data_death_long['Long'] = data_death_long[1].astype(float)

# 확진자 지도 생성
fig_confirmed = px.scatter_geo(data_confirmed_long,
                               lat='Lat',
                               lon='Long',
                               hover_name='Country',
                               size='Confirmed',
                               animation_frame='Date',
                               title='COVID-19 Confirmed Cases Over Time',
                               size_max=50)

fig_confirmed.update_layout(geo=dict(showframe=False, showcoastlines=False))
fig_confirmed.show()

# 사망자 지도 생성
fig_deaths = px.scatter_geo(data_death_long,
                            lat='Lat',
                            lon='Long',
                            hover_name='Country',
                            size='Deaths',
                            animation_frame='Date',
                            title='COVID-19 Deaths Over Time',
                            size_max=50)

fig_deaths.update_layout(geo=dict(showframe=False, showcoastlines=False))
fig_deaths.show()


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/time_series_covid_19_confirmed.csv'

In [ ]:

# datas_confirmed.rename(columns={'Country/Region':'Date'},inplace=True)
# datas_death.rename(columns={'Country/Region':'Date'},inplace=True)
# datas_confirmed
# # datas_death